# VGG8 Testing


In [1]:
import os
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np

# Imports from PyTorch.
from torch import nn, Tensor, device, no_grad, manual_seed
from torch import max as torch_max
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Imports from aihwkit.
from aihwkit.nn import AnalogConv2d, AnalogLinear, AnalogSequential
import torch 
from aihwkit.optim import AnalogSGD
from aihwkit.simulator.configs import MappingParameter
from aihwkit.simulator.rpu_base import cuda

In [ ]:
USE_CUDA = 0
if cuda.is_compiled():
    USE_CUDA = 1
DEVICE = device("cuda" if USE_CUDA else "cpu")

# Path to store datasets
PATH_DATASET = os.path.join("data", "DATASET")

# Path to store results
RESULTS = os.path.join(os.getcwd(), "results", "VGG8")

SAVE_PATH = "/u/mvc/aihwkit/notebooks/tutorial/Models/hwa_vgg8.th"
# Training parameters
SEED = 1
N_EPOCHS = 80
BATCH_SIZE = 128
LEARNING_RATE = 0.1
N_CLASSES = 10
WEIGHT_SCALING_OMEGA = 0.6  # Should not be larger than max weight.

# Select the device model to use in the training. In this case we are using one of the preset,
# but it can be changed to a number of preset to explore possible different analog devices
mapping = MappingParameter(weight_scaling_omega=WEIGHT_SCALING_OMEGA)

In [ ]:
def load_images():
    """Load images for train from torchvision datasets."""
    mean = Tensor([0.4377, 0.4438, 0.4728])
    std = Tensor([0.1980, 0.2010, 0.1970])

    print(f"Normalization data: ({mean},{std})")

    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean, std)])
    train_set = datasets.SVHN(PATH_DATASET, download=True, split="train", transform=transform)
    val_set = datasets.SVHN(PATH_DATASET, download=True, split="test", transform=transform)
    train_data = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
    validation_data = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

    return train_data, validation_data


def create_digital_network():
    """Create a Vgg8 inspired analog model.

    Returns:
       nn.Module: VGG8 model
    """
    channel_base = 48
    channel = [channel_base, 2 * channel_base, 3 * channel_base]
    fc_size = 8 * channel_base
    model = torch.nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=channel[0], kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(
            in_channels=channel[0],
            out_channels=channel[0],
            kernel_size=3,
            stride=1,
            padding=1,
        ),
        nn.BatchNorm2d(channel[0]),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1),
        nn.Conv2d(
            in_channels=channel[0],
            out_channels=channel[1],
            kernel_size=3,
            stride=1,
            padding=1,
        ),
        nn.ReLU(),
        nn.Conv2d(
            in_channels=channel[1],
            out_channels=channel[1],
            kernel_size=3,
            stride=1,
            padding=1,
        ),
        nn.BatchNorm2d(channel[1]),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1),
        nn.Conv2d(
            in_channels=channel[1],
            out_channels=channel[2],
            kernel_size=3,
            stride=1,
            padding=1,
        ),
        nn.ReLU(),
        nn.Conv2d(
            in_channels=channel[2],
            out_channels=channel[2],
            kernel_size=3,
            stride=1,
            padding=1,
        ),
        nn.BatchNorm2d(channel[2]),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1),
        nn.Flatten(),
        nn.Linear(in_features=16 * channel[2], out_features=fc_size),
        nn.ReLU(),
        nn.Linear(in_features=fc_size, out_features=N_CLASSES),
        nn.LogSoftmax(dim=1),
    )
    return model


def create_analog_network():
    """Create a Vgg8 inspired analog model.

    Returns:
       nn.Module: VGG8 model
    """
    channel_base = 48
    channel = [channel_base, 2 * channel_base, 3 * channel_base]
    fc_size = 8 * channel_base
    model = AnalogSequential(
        nn.Conv2d(in_channels=3, out_channels=channel[0], kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        AnalogConv2d(
            in_channels=channel[0],
            out_channels=channel[0],
            kernel_size=3,
            stride=1,
            padding=1,
            rpu_config=RPU_CONFIG,
        ),
        nn.BatchNorm2d(channel[0]),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1),
        AnalogConv2d(
            in_channels=channel[0],
            out_channels=channel[1],
            kernel_size=3,
            stride=1,
            padding=1,
            rpu_config=RPU_CONFIG,
        ),
        nn.ReLU(),
        AnalogConv2d(
            in_channels=channel[1],
            out_channels=channel[1],
            kernel_size=3,
            stride=1,
            padding=1,
            rpu_config=RPU_CONFIG,
        ),
        nn.BatchNorm2d(channel[1]),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1),
        AnalogConv2d(
            in_channels=channel[1],
            out_channels=channel[2],
            kernel_size=3,
            stride=1,
            padding=1,
            rpu_config=RPU_CONFIG,
        ),
        nn.ReLU(),
        AnalogConv2d(
            in_channels=channel[2],
            out_channels=channel[2],
            kernel_size=3,
            stride=1,
            padding=1,
            rpu_config=RPU_CONFIG,
        ),
        nn.BatchNorm2d(channel[2]),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1),
        nn.Flatten(),
        AnalogLinear(in_features=16 * channel[2], out_features=fc_size, rpu_config=RPU_CONFIG),
        nn.ReLU(),
        nn.Linear(in_features=fc_size, out_features=N_CLASSES),
        nn.LogSoftmax(dim=1),
    )
    return model



In [4]:
def test_evaluation(validation_data, model, criterion):
    """Test trained network

    Args:
        validation_data (DataLoader): Validation set to perform the evaluation
        model (nn.Module): Trained model to be evaluated
        criterion (nn.CrossEntropyLoss): criterion to compute loss

    Returns:
        nn.Module, float, float, float: model, test epoch loss, test error, and test accuracy
    """
    total_loss = 0
    predicted_ok = 0
    total_images = 0

    model.eval()

    for images, labels in validation_data:
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        pred = model(images)
        loss = criterion(pred, labels)
        total_loss += loss.item() * images.size(0)

        _, predicted = torch_max(pred.data, 1)
        total_images += labels.size(0)
        predicted_ok += (predicted == labels).sum().item()
        accuracy = predicted_ok / total_images * 100
        error = (1 - predicted_ok / total_images) * 100

    epoch_loss = total_loss / len(validation_data.dataset)

    return model, epoch_loss, error, accuracy

In [5]:
os.makedirs(RESULTS, exist_ok=True)
manual_seed(SEED)
from aihwkit.nn.conversion import convert_to_analog
from aihwkit.inference import ReRamCMONoiseModel
from aihwkit.simulator.parameters.io import IOParametersIRDropT

# Load datasets.
train_data, validation_data = load_images()

# Prepare the model.
model = create_digital_network()
#model.load_state_dict(torch.load("/u/mvc/aihwkit/notebooks/tutorial/Models/pre-trained-vgg8.th", map_location='cuda'))

Normalization data: (tensor([0.4377, 0.4438, 0.4728]),tensor([0.1980, 0.2010, 0.1970]))


Using downloaded and verified file: data/DATASET/train_32x32.mat
Using downloaded and verified file: data/DATASET/test_32x32.mat


In [6]:
from aihwkit.simulator.configs import (
    InferenceRPUConfig,
    NoiseManagementType,
    BoundManagementType,
)
from aihwkit.simulator.parameters.io import IOParametersIRDropT
from aihwkit.simulator.configs.utils import (
    WeightModifierType,
    BoundManagementType,
    WeightClipType,
    NoiseManagementType,
    WeightRemapType,
)
def gen_rpu_config(noise_model=None):
    input_prec = 6
    output_prec = 8
    my_rpu_config = InferenceRPUConfig()
    my_rpu_config.mapping.digital_bias = True # do the bias of the MVM digitally
    #my_rpu_config.mapping.max_input_size = 256
    #my_rpu_config.mapping.max_output_size = 256
    my_rpu_config.forward = IOParametersIRDropT()
    my_rpu_config.noise_model = noise_model
    my_rpu_config.drift_compensation = None    #my_rpu_config.noise_model = PCMLikeNoiseModel(g_max=25.0)
    #my_rpu_config.drift_compensation = GlobalDriftCompensation()
    my_rpu_config.forward.ir_drop_g_ratio = 1.0 / 0.35 / (noise_model.g_max*1e-6) # change to 25w-6 when using PCM

    #my_rpu_config.drift_compensation = None
    my_rpu_config.modifier.std_dev = 0.06
    my_rpu_config.modifier.type = WeightModifierType.ADD_NORMAL
    
    my_rpu_config.forward.inp_res = 1 / (2**input_prec - 2)
    my_rpu_config.forward.out_res = 1 / (2**output_prec - 2)
    my_rpu_config.forward.is_perfect = True
    #my_rpu_config.forward.out_noise = 0.0 # Output on the current addition (?)
    my_rpu_config.forward.ir_drop = 1.0 # TODO set to 1.0 when activating IR drop effects
    my_rpu_config.forward.ir_drop_rs = 0.35 # Default: 0.15
    my_rpu_config.pre_post.input_range.enable = True
    
    #my_rpu_config.pre_post.input_range.manage_output_clipping = True
    my_rpu_config.pre_post.input_range.decay = 0.001
    my_rpu_config.pre_post.input_range.input_min_percentage = 0.95
    my_rpu_config.pre_post.input_range.output_min_percentage = 0.95
    #my_rpu_config.forward.noise_management = NoiseManagementType.ABS_MAX # Rescale back the output with the scaling for normalizing the input
    my_rpu_config.forward.bound_management = BoundManagementType.ITERATIVE
    my_rpu_config.clip.type = WeightClipType.LAYER_GAUSSIAN
    my_rpu_config.clip.sigma = 2.5
    my_rpu_config.forward.out_bound = 10.0  # quite restric
    return my_rpu_config

In [7]:

g_max = 50
g_min = 10
prog_overshoot =1.23
single_device = False
acceptance_range = 0.2
reram_noise =ReRamCMONoiseModel(g_max=g_max, g_min=g_min,
                                                    acceptance_range=acceptance_range,
                                                    resistor_compensator=prog_overshoot,
                                                    single_device=single_device)
rpu_config= gen_rpu_config(reram_noise)
a_model = convert_to_analog(model, rpu_config=rpu_config).to(DEVICE)

In [8]:
#a_model.load_state_dict(torch.load(SAVE_PATH, map_location='cuda'))
a_model.load_state_dict(torch.load("/u/mvc/aihwkit/notebooks/tutorial/Models/hwa_2t2r_vgg8.th", map_location='cuda'))
a_model.eval()
#a_model.program_analog_weights()

/tmp/ipykernel_1045349/15031717.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  a_model.load_state_dict(torch.load("/u/mvc/aihwkit/notebooks/tutorial/Models/hwa_2t2r_vgg

AnalogSequential(
  (0): AnalogConv2d(
    3, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), InferenceRPUConfig
    (analog_module): InferenceTile(RPUCudaPulsed<float>[SimpleRPUDevice](48,27))
  )
  (1): ReLU()
  (2): AnalogConv2d(
    48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), InferenceRPUConfig
    (analog_module): InferenceTile(RPUCudaPulsed<float>[SimpleRPUDevice](48,432))
  )
  (3): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): ReLU()
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): AnalogConv2d(
    48, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), InferenceRPUConfig
    (analog_module): InferenceTile(RPUCudaPulsed<float>[SimpleRPUDevice](96,432))
  )
  (7): ReLU()
  (8): AnalogConv2d(
    96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), InferenceRPUConfig
    (analog_module): InferenceTile(RPUCudaPulsed<float>[SimpleRPUDevice](96,864))
  )
  (9): BatchNorm2

In [9]:
criterion = nn.CrossEntropyLoss()

In [10]:
n_rep = 5
t_inferences = [1, 60*10, 3600, 3600 * 24, 3600 * 24*7, 3600 * 24 *30, 3600 * 24 *365, 3600 * 24 *365*2, 3600 * 24 *365*5, 3600 * 24 * 365 * 10]
#t_inferences = [ 3600 * 24]
drifted_test_accs = torch.zeros(size=(len(t_inferences),n_rep))

for i,t in enumerate(t_inferences):
    for j in range(n_rep):
        a_model.drift_analog_weights(t)
        print("Drifted at t: ", t)
        _,_,err, accuracy = test_evaluation(validation_data=validation_data, model=a_model, criterion=criterion)
        drifted_test_accs[i, j] = accuracy
        print(f"Accuracy of the analog model: {accuracy:.2f}%")

Drifted at t:  1
Accuracy of the analog model: 94.33%
Drifted at t:  1
Accuracy of the analog model: 94.34%
Drifted at t:  1
Accuracy of the analog model: 94.33%
Drifted at t:  1
Accuracy of the analog model: 94.32%
Drifted at t:  1
Accuracy of the analog model: 94.33%
Drifted at t:  600
Accuracy of the analog model: 94.43%
Drifted at t:  600
Accuracy of the analog model: 94.38%
Drifted at t:  600
Accuracy of the analog model: 94.41%
Drifted at t:  600
Accuracy of the analog model: 94.51%
Drifted at t:  600
Accuracy of the analog model: 94.53%
Drifted at t:  3600
Accuracy of the analog model: 94.48%
Drifted at t:  3600
Accuracy of the analog model: 94.48%
Drifted at t:  3600
Accuracy of the analog model: 94.43%
Drifted at t:  3600
Accuracy of the analog model: 94.51%
Drifted at t:  3600
Accuracy of the analog model: 94.45%
Drifted at t:  86400
Accuracy of the analog model: 94.54%
Drifted at t:  86400
Accuracy of the analog model: 94.56%
Drifted at t:  86400
Accuracy of the analog model

In [11]:
# BERT on SQUAD

In [11]:
torch.save(drifted_test_accs, "/u/mvc/aihwkit/drift_compensation_NNs/hwa_vgg8_2T2R_10-50_prog.th")